In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import time

2024-10-04 18:03:01.982068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 18:03:02.005488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 18:03:02.014820: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 18:03:02.030488: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-04 18:03:03.713007: W tensorflow/compiler/tf2

In [2]:
seed = 42

In [3]:
dataset = np.load('../data/dataset.npz',)
X, y = dataset['X'], dataset['y']
print(X.shape, y.shape)

(9834, 400) (9834,)


In [4]:
# fig, axis = plt.subplots(17, 5, figsize=(12, 48))

# for label in range(17):
#     for i in range(5):
#         axis[label, i].imshow(X[y == label][i+5].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()

In [5]:
for i in range(17):
    print(f'Label {i}: {len(X[y == i])}')

max([len(X[y == i]) for i in range(17)]) * 17


Label 0: 627
Label 1: 230
Label 2: 525
Label 3: 950
Label 4: 500
Label 5: 695
Label 6: 912
Label 7: 605
Label 8: 427
Label 9: 205
Label 10: 825
Label 11: 525
Label 12: 950
Label 13: 909
Label 14: 74
Label 15: 450
Label 16: 425


16150

#### SMOTE

In [6]:
# sm = SMOTE(random_state=seed, k_neighbors=2)
# X_res, y_res = sm.fit_resample(X.reshape(X.shape[0], -1), y)
# print(X_res.shape, y_res.shape)

# #fig, axis = plt.subplots(17, 5, figsize=(12, 48))
# for label in range(17):
#     for i in range(5):
#         axis[label, i].imshow(X_res[y_res == label][949-i].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()

In [7]:
def generate_label(X, y, label, n):
    """generates n augmented images for a given label"""
    X = X[y == label]
    y = y[y == label]
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'
    )
    X_reshaped = X.reshape(X.shape[0], 20, 20, 1)

    augmented_data = datagen.flow(X_reshaped, y, batch_size=1, seed=seed)
    X_augs, y_augs = [], []
    for i in range(n):
        X_aug, y_aug = augmented_data.__next__()
        X_aug = X_aug.flatten()
        X_augs.append(X_aug)
        y_augs.append(y_aug)
    
    X_augs = np.array(X_augs)
    y_augs = np.array(y_augs).reshape(-1)

    return np.array(X_augs), np.array(y_augs)

print(X.shape, y.shape)
X_aug, y_aug = generate_label(X, y, 1, 5)
print(X_aug.shape, y_aug.shape)

(9834, 400) (9834,)
(5, 400) (5,)


In [8]:
from preprocessing import generate_balanced_data

X_aug, y_aug = generate_balanced_data(X, y, 42)

print(X_aug.shape, y_aug.shape)
# fig, axis = plt.subplots(17, 5, figsize=(12, 48))

# for label in range(0,17):
#     for i in range(5):
#         axis[label, i].imshow(X_aug[y_aug == label][i].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()

(16150, 400) (16150,)


In [9]:
dataset = np.load('../data/corrupt_dataset.npz',)
CX = dataset['X']
print(CX.shape)
# fig, axis = plt.subplots(17, 5, figsize=(12, 48))

# for label in range(0,17):
#     for i in range(5):
#         axis[label, i].imshow(X[i+label+5].reshape(20, 20), vmin=0, vmax=255, cmap='gray')
#         axis[label, i].set_xticks([])
#         axis[label, i].set_yticks([])
#         axis[label, i].set_title(f'Label {label}')

# plt.show()


(935, 400)


In [10]:
class LeNet(nn.Module):
    def __init__(self, numChannels, classes):
        super(LeNet, self).__init__()

        #scales it down to 18x18 x 20
        self.conv1 = nn.Conv2d(
            in_channels=numChannels, 
            out_channels=20,
            kernel_size=(3,3), 
            )
        
        #first relu pass
        self.relu1 = nn.ReLU()
        
        #scales it down to 9x9 x 20
        self.maxpool1 = nn.MaxPool2d(
            kernel_size=(2,2),
            stride=(2,2)
            )

        #scales it down to 7x7 x 50 
        self.conv2 = nn.Conv2d(
            in_channels=20, 
            out_channels=50,
            kernel_size=(3,3), 
            )

        #second relu pass
        self.relu2 = nn.ReLU()

        #scales it down to 3x3 x 50
        self.maxpool2 = nn.MaxPool2d(
            kernel_size=(2,2),
            stride=(2,2),
            )

        #takes the 3x3x50 = 450
        self.fc1 = nn.Linear(
            in_features=450,
            out_features=500,
            )
        
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(
            in_features=500,
            out_features=classes,
        )
        self.logsoftmax = nn.LogSoftmax(dim=1)


    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)

        x = self.fc2(x)
        output = self.logsoftmax(x)
        return output
            

In [11]:
INIT_LR = 1e-3
BATCH_SIZE = 32
EPOCHS = 10

TRAIN_SPLIT = 0.70
TEST_SPLIT = 1 - TRAIN_SPLIT

device = torch.device("cpu")

In [12]:
X_train, X_val_test, y_train, y_val_test = train_test_split(
    X_aug, 
    y_aug, 
    test_size=TEST_SPLIT, 
    random_state=seed
    )

X_val, X_test, y_val, y_test = train_test_split(
    X_val_test, 
    y_val_test, 
    test_size=0.5, 
    random_state=seed
    )

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
    )

val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
    )

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
    )

model = LeNet(numChannels=1, classes=17).to(device)
optimizer = optim.Adam(model.parameters(), lr=INIT_LR)
criterion = nn.NLLLoss()

H = {
    'train_loss': [],
    'val_loss': [],
    'train_acc': [],
    'val_acc': []
}

print("training the network now")

for epoch in range(EPOCHS):
    model.train()

    total_train_loss = 0
    total_val_loss = 0

    correct_train = 0
    correct_val = 0

    for i, (X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        X_batch = X_batch.view(X_batch.size(0), 1, 20, 20)  # Reshape the input tensor
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        correct_train += (y_pred.argmax(1) == y_batch).type(torch.float).sum().item()

    with torch.no_grad():
        model.eval()

        for X_val_batch, y_val_batch in val_loader:
            X_val_batch = X_val_batch.view(X_val_batch.size(0), 1, 20, 20)
            y_val_pred = model(X_val_batch)
            loss = criterion(y_val_pred, y_val_batch)
            total_val_loss += loss.item()
            correct_val += (y_val_pred.argmax(1) == y_val_batch).type(torch.float).sum().item()
        
    
    avg_train_loss = total_train_loss / len(train_loader.dataset)
    avg_val_loss = total_val_loss / len(val_loader.dataset)
    train_acc = correct_train / len(train_loader.dataset)
    val_acc = correct_val / len(val_loader.dataset)

    H['train_loss'].append(avg_train_loss)
    H['val_loss'].append(avg_val_loss)
    H['train_acc'].append(train_acc)
    H['val_acc'].append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train loss: {avg_train_loss:.4f}, Train accuracy: {train_acc:.4f}")
    print(f"Val loss: {avg_val_loss:.4f}, Val accuracy: {val_acc:.4f}")

print("training complete")



training the network now
Epoch 1/10
Train loss: 0.0448, Train accuracy: 0.6831
Val loss: 0.0150, Val accuracy: 0.8469
Epoch 2/10
Train loss: 0.0117, Train accuracy: 0.8796
Val loss: 0.0122, Val accuracy: 0.8803
Epoch 3/10
Train loss: 0.0079, Train accuracy: 0.9194
Val loss: 0.0104, Val accuracy: 0.8993
Epoch 4/10
Train loss: 0.0059, Train accuracy: 0.9373
Val loss: 0.0085, Val accuracy: 0.9154
Epoch 5/10
Train loss: 0.0042, Train accuracy: 0.9532
Val loss: 0.0093, Val accuracy: 0.9137
Epoch 6/10
Train loss: 0.0037, Train accuracy: 0.9612
Val loss: 0.0084, Val accuracy: 0.9261
Epoch 7/10
Train loss: 0.0031, Train accuracy: 0.9651
Val loss: 0.0087, Val accuracy: 0.9249
Epoch 8/10
Train loss: 0.0026, Train accuracy: 0.9710
Val loss: 0.0099, Val accuracy: 0.9179
Epoch 9/10
Train loss: 0.0027, Train accuracy: 0.9716
Val loss: 0.0083, Val accuracy: 0.9331
Epoch 10/10
Train loss: 0.0020, Train accuracy: 0.9775
Val loss: 0.0104, Val accuracy: 0.9241
training complete


In [13]:
with torch.no_grad():
    model.eval()
    preds = []
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.view(X_batch.size(0), 1, 20, 20)
        y_pred = model(X_batch)
        preds.append(y_pred.argmax(1))

    preds = torch.cat(preds).cpu().numpy()

print(classification_report(y_test.cpu().numpy(), preds))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       140
           1       0.86      1.00      0.92       136
           2       0.89      0.94      0.91       146
           3       0.99      0.90      0.95       164
           4       0.94      0.93      0.93       153
           5       0.92      0.95      0.94       138
           6       0.99      0.97      0.98       145
           7       0.92      0.96      0.94       137
           8       0.90      0.84      0.87       129
           9       0.92      0.92      0.92       147
          10       0.90      0.93      0.91       157
          11       0.88      0.91      0.90       151
          12       0.96      0.98      0.97       137
          13       0.95      0.87      0.91       132
          14       0.95      0.81      0.87       128
          15       0.89      0.85      0.87       139
          16       1.00      1.00      1.00       144

    accuracy              